# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Estudo de Caso - Agregação e Sumarização com MapReduce e PySpark</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


### *********** Atenção: *********** 
Utilize Java JDK 1.8 ou 11 e Apache Spark 2.4.2 ou 2.4.5

## Definição do Problema

A definição do problema para este estudo de caso está no manual em pdf onde você encontrou este Jupyter Notebook.

Este estudo de caso é mais um recurso de aprendizado adicional fornecido pela Data Science Academy. Aproveite.

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Imports
import time
import argparse
import pyspark
import numpy as np 
import pandas as pd
import datetime as dt
from pathlib import Path
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

platform 1.0.8
numpy    1.18.2
py4j     0.10.7
pyspark  2.4.5
argparse 1.1
pandas   1.0.3
Data Science Academy


In [5]:
# Cria sessão Spark
spark = SparkSession(sc)

In [6]:
# Define o arquivo
arquivo = 'dados/dataset.csv'

In [7]:
# Carrega os dados
dataset = pd.read_csv(arquivo)

In [8]:
# Visualiza os dados
dataset.head()

Genero_Usuario  Idade_Usuario   Bike  Estacao_Aluguel Data_Aluguel  \
0              M             44   4357              442   01/02/2020   
1              M             22  12083               66   01/02/2020   
2              M             29  11562              331   01/02/2020   
3              M             27  10206              164   01/02/2020   
4              M             27  10101              120   01/02/2020   

  Hora_Aluguel  Estacao_Chegada Data_Chegada Hora_Chegada  
0      0:00:38              116   01/02/2020      0:35:17  
1      0:00:53               37   01/02/2020      0:06:23  
2      0:00:55              341   01/02/2020      0:26:47  
3      0:01:18               35   01/02/2020      0:16:51  
4      0:01:18               47   01/02/2020      0:12:39

In [9]:
# Shape
dataset.shape

(686327, 9)

In [10]:
# Tipos de dados
dataset.dtypes

Genero_Usuario     object
Idade_Usuario       int64
Bike                int64
Estacao_Aluguel     int64
Data_Aluguel       object
Hora_Aluguel       object
Estacao_Chegada     int64
Data_Chegada       object
Hora_Chegada       object
dtype: object

### Função Para Obter a Data

In [11]:
# Função para obter a data a partir do dataset
def gera_data(date, time):
    from datetime import datetime    
    datetime_object = datetime.strptime(date + " " + time, '%d/%m/%Y %H:%M:%S')
    return datetime_object

### Função Para Criar Faixas Etárias

In [12]:
# Função para obter faixa etária
def gera_faixa_etaria(idade):
    if idade < 18:
        return "00-18"
    elif idade >= 18 and idade <= 34:
        return "18-34"
    elif idade >= 35 and idade <= 44:
        return "35-44"
    elif idade >= 45 and idade <= 54:
        return "45-54"
    elif idade >= 55 and idade <= 64:
        return "55-64"
    else:
        return "65+"

### Função Para Limpeza dos Dados

In [13]:
# Função para limpeza de dados
def limpa_dados(part_id, list_of_records):
    if part_id == 0: 
        next(list_of_records) 
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        gender = row[0]
        idade = int(row[1])
        bike_id = int(row[2])
        station_start = row[3]
        datetime_start = gera_data(row[4], row[5])
        datetime_end = gera_data(row[7], row[8])
        station_end = row[6]
        yield (bike_id, gender, gera_faixa_etaria(idade), station_start, station_end, datetime_start, datetime_end)

### Função Para Gerar RDDs

In [14]:
# Função para gerar RDDs
def gera_rdd(sc, input_file):
    print("Lendo o Arquivo de Dados:", input_file)

    # Gera RDDs a partir do arquivo
    rides_rdd = sc.textFile(input_file, use_unicode = True).mapPartitionsWithIndex(limpa_dados).cache()
    
    print("Número de Partições: ", rides_rdd.getNumPartitions())
    return rides_rdd

### Função Para Calcular as Top Start Stations

In [15]:
# Função Para Calcular as Top Stations
def calcula_top_start_stations(num, rides_rdd):
    results = rides_rdd.map(lambda x: (x[3], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x:  1 ) \
        .reduceByKey(lambda x,y: x+y) \
        .map(lambda x: (x[1], x[0])) \
        .top(num, key=lambda x: x) 
    return results

### Função Para Calcular as Principais Rotas

In [16]:
# Função Para Calcular as Principais Rotas
def calcula_top_rotas(num, rides_rdd): 
    results = rides_rdd.map(lambda x: ( (x[3],x[4]), x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x,y: ( x[0] + y[0], x[1] + y[1] ) ) \
        .map(lambda x: (x[1][1], ( x[0], x[1][0]/x[1][1] ) ) ) \
        .top(num, key = lambda x: x)   
    return results

### Função Para Calcular Estações Por Genero

In [17]:
# Função Para Calcular Estações Por Genero
def calcula_stats_genero(rides_rdd):
    results = rides_rdd.map(lambda x: (x[1], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .mapValues(lambda x: (x[0] / x[1], x[1]) ) \
        .collect()
    return results 

### Função Para Calcular Estações Por Idade

In [18]:
# Função Para Calcular Estações Por Idade
def calcula_stats_idade(rides_rdd):
    results = rides_rdd.map(lambda x: (x[2], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .mapValues(lambda x: (x[0] / x[1], x[1]) ) \
        .collect()
    return results  

### Função Para Calcular Estações Com Mais Movimento

In [19]:
# Função Para Calcular Estações Com Mais Movimento
def calcula_stats_busy(num, rides_rdd, by_count = False):
    results = rides_rdd.map(lambda x: (x[0], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (1, x.total_seconds())) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .map(lambda x: (x[1], x[0])) 
    
    if by_count: 
        return results.top(num, key = lambda x: x[0][0])
    else: 
        return results.top(num, key = lambda x: x[0][1])   

### Executando as Funções e Obtendo as Respostas

In [20]:
print("\nGerando RDD Para Todas as Corridas de Bike...", arquivo)
rides_rdd = gera_rdd(sc, arquivo)


Gerando RDD Para Todas as Corridas de Bike... dados/dataset.csv
Lendo o Arquivo de Dados: dados/dataset.csv
Número de Partições:  2


### 1. Quais são as Top 5 estações com maior número de aluguel de bikes?

In [21]:
top_stations = calcula_top_start_stations(5, rides_rdd)
print("Top 5 Estações:\n")
for entry in top_stations:
    print("Estacões: {:03d}, Número de Bikes Alugadas: {:03d}".format(int(entry[1]), entry[0]))

Top 5 Estações:

Estacões: 001, Número de Bikes Alugadas: 6298
Estacões: 027, Número de Bikes Alugadas: 6201
Estacões: 271, Número de Bikes Alugadas: 5262
Estacões: 064, Número de Bikes Alugadas: 4825
Estacões: 041, Número de Bikes Alugadas: 4621


### 2. Quais são as Top 5 rotas, com base na estação inicial e final, e a média de duração de cada aluguel?

In [22]:
top_routes = calcula_top_rotas(5, rides_rdd)
print("Top 5 Rotas de Bikes:\n")
for entry in top_routes:
    print("Da Estação: {:03d}, Para a Estação: {:03d}, Número Total de Bikes Alugadas: {:03d}, Duração Média(mins): {:.2f}".format(
        int(entry[1][0][0]), int(entry[1][0][1]), entry[0], entry[1][1]/60))

Top 5 Rotas de Bikes:

Da Estação: 033, Para a Estação: 033, Número Total de Bikes Alugadas: 375, Duração Média(mins): 30.23
Da Estação: 018, Para a Estação: 001, Número Total de Bikes Alugadas: 319, Duração Média(mins): 5.58
Da Estação: 211, Para a Estação: 217, Número Total de Bikes Alugadas: 303, Duração Média(mins): 3.54
Da Estação: 449, Para a Estação: 449, Número Total de Bikes Alugadas: 301, Duração Média(mins): 15.34
Da Estação: 208, Para a Estação: 206, Número Total de Bikes Alugadas: 297, Duração Média(mins): 8.53


### 3. Quem aluga mais bikes, homens ou mulheres? Qual o tempo médio de aluguel de bikes?

In [23]:
gender_stats = calcula_stats_genero(rides_rdd)
print("Perfil de Aluguel de Bikes Por Genero:\n")
for entry in gender_stats:
    print("Genero: {}, Total: {}, Tempo Médio de Aluguel(mins): {:.2f}".format(entry[0], entry[1][1], entry[1][0]/60 ))

Perfil de Aluguel de Bikes Por Genero:

Genero: M, Total: 509782, Tempo Médio de Aluguel(mins): 13.62
Genero: F, Total: 174808, Tempo Médio de Aluguel(mins): 14.30


### 4. Qual faixa etária aluga mais bikes? Qual o tempo médio de aluguel de bikes?

In [24]:
age_stats = calcula_stats_idade(rides_rdd)
print("Perfil de Aluguel de Bikes Por Idade:\n")
for entry in age_stats:
    print("Idade: {}, Total: {:06d}, Tempo Médio de Aluguel(mins): {:.2f}".format(entry[0], entry[1][1], entry[1][0]/60 ))

Perfil de Aluguel de Bikes Por Idade:

Idade: 35-44, Total: 167715, Tempo Médio de Aluguel(mins): 13.58
Idade: 18-34, Total: 392040, Tempo Médio de Aluguel(mins): 14.03
Idade: 45-54, Total: 077990, Tempo Médio de Aluguel(mins): 13.30
Idade: 55-64, Total: 035831, Tempo Médio de Aluguel(mins): 13.30
Idade: 00-18, Total: 001332, Tempo Médio de Aluguel(mins): 12.78
Idade: 65+, Total: 009682, Tempo Médio de Aluguel(mins): 13.69


### 5. Quais são as estações com maior número de bikes alugadas/devolvidas?

In [25]:
most_used_bikes = calcula_stats_busy(3, rides_rdd, True)
print("Estações Com Maior Número de Bikes Alugadas/Devolvidas:\n")
for entry in most_used_bikes: 
    print("ID Estação: {:05d}, Total: {}, Minutos: {:.2f}".format(entry[1], entry[0][0], entry[0][1]/60 ))

Estações Com Maior Número de Bikes Alugadas/Devolvidas:

ID Estação: 10771, Total: 217, Minutos: 1523.15
ID Estação: 10810, Total: 208, Minutos: 2785.77
ID Estação: 07854, Total: 192, Minutos: 2633.07


# Fim